# 01. Decennial Census 2010 - 2020
### Pop & Housing variables pulled from 2010, and 2020 Censuses (using Cen API) for urban villages,  City of Phoenix and U.S.

Total Housing Units, Population, Population by Ethncity, Total Occupied Housing Units,  


In [1]:
import pandas as pd
import numpy as np

In [2]:
from geo import *
import get_census as get

In [3]:
source_dec = 'dec/sf1'
source_red = 'dec/pl'
years = {'2010':'P001001,P005003,P005004,P005005,P005006,P005007,P005008,P005009,P005010',\
        '2020':'P1_001N,P2_002N,P2_005N,P2_006N,P2_007N,P2_008N,P2_009N,P2_010N,P2_011N,H1_001N'}

col_10_rename={'P001001':'Pop_10E','P005003':'P_Wh_10E','P005004':'P_Bl_10E','P005006':'P_As_10E','P005010':'P_Hi_10E',\
               'P005005':'P_Ot_10E','P005007':'P_Ot_10E','P005008':'P_Ot_10E','P005009':'P_Ot_10E'}
col_20_rename = {'P1_001N':'Pop_20E','P2_002N':'P_Hi_20E',\
              'P2_005N':'P_Wh_20E','P2_006N':'P_Bl_20E',\
              'P2_007N':'P_Ot_20E','P2_008N':'P_As_20E','P2_009N':'P_Ot_20E',\
              'P2_010N':'P_Ot_20E','P2_011N':'P_Ot_20E','H1_001N':'Hou_20E'}

### build county, subregion, region table for 2000, 2010 and 2020

In [4]:
co00 = get.get_co_dec(source_dec,list(years.keys())[0],years.get(list(years.keys())[0]),col_00_rename)
co10 = get.get_co_dec(source_dec,list(years.keys())[1],years.get(list(years.keys())[1]),col_10_rename)
co20 = get.get_co_dec(source_red,list(years.keys())[2],years.get(list(years.keys())[2]),col_20_rename)
labels = ['id','sub']
co = pd.merge(pd.merge(co00,co10,on=labels,how='left'),co20,on=labels,how='left')

#make sub & region
sub = pd.pivot_table(co,values=co.columns[len(labels):],index=['sub'],\
                            aggfunc=np.sum,fill_value=0,margins=True).reset_index()
sub.loc[sub['sub']=='All',['sub']] = 'METRO'
sub = sub.rename(columns={'sub':'id'})
co = co.drop(columns='sub')
cosubreg = pd.concat([co,sub])

In [6]:
cosubreg.to_csv('../output/intermediate/dec_csr.csv',index=False)

### build municipality table 2010 & 2020 only

In [7]:
nyc_10 = get.get_nycsub_dec(source_dec,list(years.keys())[1],years.get(list(years.keys())[1]),col_10_rename)
nyc_20 = get.get_nycsub_dec(source_red,list(years.keys())[2],years.get(list(years.keys())[2]),col_20_rename)
nyc = pd.merge(nyc_10,nyc_20,on='id',how='left')

subpl_10 = get.get_subpl_dec(source_dec,list(years.keys())[1],years.get(list(years.keys())[1]),col_10_rename)
subpl_20 = get.get_subpl_dec(source_red,list(years.keys())[2],years.get(list(years.keys())[2]),col_20_rename)
subpl = pd.merge(subpl_10,subpl_20,on='id',how='left')

In [10]:
muni = pd.concat([nyc,subpl])
muni['id'] = muni['id'].astype(str)
muni = muni[~muni['id'].isin(li_nyc_sub)]

In [12]:
muni.to_csv('../output/intermediate/dec_m.csv',index=False)

### build tract table for 2010 and 2020 - race/hispanic origin (for mapping)

In [13]:
tract_10 = get.get_tract_dec(source_dec,list(years.keys())[1],years.get(list(years.keys())[1]),col_10_rename)
tract_20 = get.get_tract_dec(source_red,list(years.keys())[2],years.get(list(years.keys())[2]),col_20_rename)

In [14]:
tract_10.to_csv('../output/intermediate/race10_t.csv',index=False)
tract_20.to_csv('../output/intermediate/race20_t.csv',index=False)